In [158]:
import json
from datetime import datetime
import plotly.express as px
from sklearn.model_selection import train_test_split
import sqlite3
import nltk
import spacy
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import umap
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
%matplotlib inline

In [264]:
df_reviews_4yrs_open = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_reviews_last_4yrs_open.csv')



KeyboardInterrupt: 

In [159]:
df_business_last_4yrs = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_business_last_4yrs_open.csv')


In [308]:
df_business_last_4yrs['review_count'].value_counts()

3       25906
4       14132
5       10986
6        8852
7        7430
        ...  
1445        1
2648        1
3414        1
2855        1
1791        1
Name: review_count, Length: 1292, dtype: int64

In [310]:
df_bus_select =  df_business_last_4yrs[df_business_last_4yrs['review_count']>100]

In [311]:
df_bus_select['review_count'].value_counts()

105     144
107     144
113     137
102     133
103     129
       ... 
1827      1
1803      1
1747      1
1675      1
4138      1
Name: review_count, Length: 1194, dtype: int64

In [280]:
threshold = 0.2
review_by_state = pd.DataFrame(df_bus_select['state'].value_counts())
states_select = review_by_state[review_by_state['state'] > threshold*review_by_state.max()['state']]
states_select

,state
AZ,4664
NV,4459
ON,1508
NC,966


In [281]:
df_reviews_4yrs_open.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464626 entries, 0 to 4464625
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Unnamed: 0   object 
 1   review_id    object 
 2   user_id      object 
 3   business_id  object 
 4   stars        float64
 5   useful       float64
 6   funny        float64
 7   cool         float64
 8   text         object 
 9   date         object 
 10  year         float64
dtypes: float64(5), object(6)
memory usage: 374.7+ MB


In [283]:
df_rev_bus_raw = df_reviews_4yrs.merge(df_bus_select, on = 'business_id', how= 'inner')

In [293]:
df_rev_bus_raw.shape

(2617026, 25)

In [284]:
df_rev_bus_raw = df_rev_bus_raw.rename(columns = {'stars_x': 'stars'})

In [285]:
df_rev_bus_select = df_rev_bus_raw.loc[df_rev_bus_raw['state'].isin(states_select.index)==True]

In [286]:
df_rev_bus_select.state.value_counts()

NV    1146452
AZ     837743
ON     200238
NC     156578
Name: state, dtype: int64

In [287]:
df_food_rev = df_rev_bus_select[df_rev_bus_select['categories'].str.contains('Restaurant')|
                 df_rev_bus_select['categories'].str.contains('Coffee & Tea')]

In [288]:
df_food = df_food_rev.copy()
catlist_clean = df_food['categories'].str.replace(' ', '' ).str.replace(',', ', ')

In [289]:
df_food['categories_filtered']= catlist_clean

In [290]:
food_categories = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/Yelp_food_categories.csv')
food_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Categories   0 non-null      float64
 1   Food_types   69 non-null     object 
 2   Ethnicities  96 non-null     object 
 3   Venue_type   51 non-null     object 
dtypes: float64(1), object(3)
memory usage: 6.9+ KB


In [291]:
food_types = food_categories['Food_types'].dropna().str.strip().str.replace(' ', '')
ethnicities = food_categories['Ethnicities'].dropna().str.strip().str.replace(' ', '')
venue_types = food_categories['Venue_type'].dropna().str.strip().str.replace(' ', '')

In [292]:
type_list = []

for x in df_food['categories_filtered']:
    sublist = []
    for i in food_types:
        if i in x:
            sublist.append(i)
    
    if len(sublist) == 0:
        type_list.append(['Undefined'])
    else:
        type_list.append(sublist)  

print(len(type_list))
print(df_food.shape[0])


1833249
1833249


In [294]:
venue_list = []

for x in  df_food['categories_filtered']:
    sublist = []
    for i in venue_types:
        if i in x:
            sublist.append(i)
    
    if len(sublist) == 0:
        venue_list.append(['Restaurant'])
    else:
        venue_list.append(sublist)  

print(len(venue_list))
print(df_food.shape[0])


1833249
1833249


In [295]:
ethnicity_list = []

for x in  df_food['categories_filtered']:
    sublist = []
    for i in ethnicities:
        if i in x:
            sublist.append(i)
    
    if len(sublist) == 0:
        ethnicity_list.append(['Undefined'])
    else:
        ethnicity_list.append(sublist)  
print(len(ethnicity_list))
print(df_food.shape[0])


1833249
1833249


In [296]:
df_food['food_type'] = type_list
df_food['ethnicity'] = ethnicity_list
df_food['venue'] = venue_list


In [297]:
df_food['food_comb'] = (df_food['food_type']+df_food['ethnicity']+df_food['venue']).apply(lambda x: ', '.join(x))
df_food['food_comb'] = df_food['food_comb'].str.replace('Undefined,', '')


In [298]:
df_food_clean = df_food.drop(columns = ['categories'])
df_food_clean['food_type'] = df_food_clean['food_type'].apply(lambda x: ', '.join(x))
df_food_clean['venue'] = df_food_clean['venue'].apply(lambda x: ', '.join(x))
df_food_clean['ethnicity']= df_food_clean['ethnicity'].apply(lambda x: ', '.join(x))

In [299]:
(df_food_clean['postal_code']=='').value_counts()

False    1833249
Name: postal_code, dtype: int64

In [300]:
df_food_clean['postal_code'].replace('', np.nan, inplace = True)
df_food_clean.dropna(inplace = True)

In [301]:
df_food_clean.reset_index(drop=True, inplace= True)


In [302]:
df_food_clean = df_food_clean.rename(columns = {'stars_x':'stars'})

In [303]:
df_food_clean

,Unnamed: 0_x,review_id,user_id,business_id,stars,useful,funny,cool,text,date,...,stars_y,review_count,is_open,attributes,hours,categories_filtered,food_type,ethnicity,venue,food_comb
0,4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,0.0,0.0,0.0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
1,3103,gQuXBrYo8dqY7h_rfDzwLw,46TNf-5T5EcNFRJPYv_nrw,IS4cv902ykd8wj1TR0N3-A,4.0,0.0,0.0,0.0,Love the chicken. It's always hot and fresh ta...,2018-01-09 20:25:20,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
2,3173,xkq0ekkZ39uSbb29LihkfQ,OTGWjBCELxkZ6lpS2OEtxQ,IS4cv902ykd8wj1TR0N3-A,3.0,0.0,1.0,0.0,"Great food right down the street, long line si...",2017-01-28 00:03:26,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
3,20520,m4AkgxXvb-GHuDK4b9i9sA,YvlcRfeulgtusMFBOQTWzw,IS4cv902ykd8wj1TR0N3-A,1.0,0.0,0.0,1.0,Terrible terrible service I waited 30 minutes ...,2017-01-15 05:55:07,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
4,22003,OkgQVoHBJ6w-4qL-9lOczA,DBq1YkbIJ4yTXUYJw1V_KA,IS4cv902ykd8wj1TR0N3-A,3.0,0.0,0.0,0.0,I am giving it 3 stars because all the drinks ...,2017-03-15 21:36:27,...,3.5,159,1,"{'DriveThru': 'True', 'GoodForMeal': ""{'desser...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","FastFood, Restaurants, Sandwiches, ChickenWings","ChickenWings, FastFood, Sandwiches",Undefined,Restaurant,"ChickenWings, FastFood, Sandwiches, Restaurant"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814878,8009356,Zg3NGfTXJcmi_ZFnugrXdQ,hD9Hr15XgM_NTAsT7aboHg,ZrHFTAckri4Iv9z55NDD_w,5.0,0.0,0.0,0.0,This place reminds my husband and me about a r...,2019-07-01 02:04:11,...,4.0,125,1,"{'GoodForMeal': ""{'dessert': False, 'latenight...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ...","Japanese, JapaneseCurry, Ramen, Restaurants","JapaneseCurry, Ramen",Japanese,Restaurant,"JapaneseCurry, Ramen, Japanese, Restaurant"
1814879,8016256,b1u-gqS9z_QD1AInjnGmzA,PnaEwJPxwuYVIO269NZE0A,ZrHFTAckri4Iv9z55NDD_w,5.0,0.0,0.0,1.0,Who doesn't love curry!? I really enjoyed my e...,2019-10-15 22:18:00,...,4.0,125,1,"{'GoodForMeal': ""{'dessert': False, 'latenight...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ...","Japanese, JapaneseCurry, Ramen, Restaurants","JapaneseCurry, Ramen",Japanese,Restaurant,"JapaneseCurry, Ramen, Japanese, Restaurant"
1814880,8017474,WyyMWmG_huNZ0g4TQtqy4A,TsUP8OWLOJu8C4gGmrnctQ,ZrHFTAckri4Iv9z55NDD_w,4.0,0.0,0.0,0.0,I came in because I was craving takoyaki and t...,2019-11-23 05:37:35,...,4.0,125,1,"{'GoodForMeal': ""{'dessert': False, 'latenight...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ...","Japanese, JapaneseCurry, Ramen, Restaurants","JapaneseCurry, Ramen",Japanese,Restaurant,"JapaneseCurry, Ramen, Japanese, Restaurant"
1814881,8017497,ykj8iOY27SWS10dK5yeYkw,SS3sFA9ksCT9bjocM3Wbug,ZrHFTAckri4Iv9z55NDD_w,1.0,1.0,1.0,0.0,Just found out they won't substitute the chick...,2019-11-26 04:59:11,...,4.0,125,1,"{'GoodForMeal': ""{'dessert': False, 'latenight...","{'Monday': '0:0-0:0', 'T

In [304]:
df_food_clean.drop(['Unnamed: 0_x'], axis = True, inplace = True)

In [305]:
df_food_clean.to_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_food_clean_rev.csv')

In [307]:
df_food_clean['review_count'].value_counts()

10129    6052
107      5507
113      5434
198      5374
143      5224
         ... 
609       243
777       235
718       220
564       220
494       216
Name: review_count, Length: 1114, dtype: int64